In [2]:
import geopandas as gpd
import fiona
import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import configparser
import momepy
import libpysal
import pandas as pd
import osmnx
import pandas
from bokeh.io import output_notebook
from bokeh.plotting import show
from clustergram import Clustergram
from shapely.geometry import Point
local_crs = "EPSG:2039"
output_notebook()

Loading BokehJS ...

Load all the gdb files from michael folders

In [3]:

config = configparser.ConfigParser()
config_file = 'config.ini'

# check if file exists
if not os.path.isfile(config_file):
    # create a simple config file
    config['Paths'] = {'root_folder': ''}
    with open(config_file, 'w') as configfile:
        config.write(configfile)
    print(f"Created configuration file '{config_file}'.")

config.read('config.ini')

root_folder = config['Paths']['root_folder']  
print(root_folder)
root_folder = os.path.join(root_folder, "Michaels_Data")
print(root_folder)
# List all .gdb directories in the folder
gdb_files = []
for dirpath, dirnames, filenames in os.walk(root_folder):
    # Check if the folder contains a "commondata" folder
    if "commondata" in dirnames:
        commondata_folder = os.path.join(dirpath, "commondata")
        commondata_folder = os.path.normpath(commondata_folder)

        # List all .gdb files in the "commondata" folder
        for f in os.listdir(commondata_folder):
            if f.endswith('.gdb'):
                gdb_files.append(os.path.join(commondata_folder, f))

# Print the collected .gdb files
for gdb in gdb_files:
    print(gdb)
print("GDB Files:", gdb_files)


Jerusaelm_data
GDB Files: []


Calculating metrics over buildings:

In [4]:
# load municipal buildings
gdb_file = gdb_files[4]  # Modify if you want to choose a different .gdb
print(gdb_file)

# List the layers in the selected .gdb
layers = fiona.listlayers(gdb_file)
print("Layers in the selected GDB:", layers)

# Choose a specific layer within the .gdb
textures_layer = layers[0]  # Modify if you want to choose a different layer

# Load the specific layer
gdf = gpd.read_file(gdb_file, layer=textures_layer)

IndexError: list index out of range

In [4]:
# preprocess the buildings
from sklearn.preprocessing import LabelEncoder
buildings = gdf.to_crs(local_crs)

# remove the duplicates
duplicates = buildings[buildings.duplicated(subset='geometry', keep=False)]
buildings = buildings.drop_duplicates(subset='geometry', keep='first')
buildings = buildings.reset_index(drop=True)
buildings = buildings.drop(columns=['BLDG_TYPE','BLDG_CH','StreetNa_1'])

# Assuming `buildings` is your DataFrame and `categorical_column` is the column you want to encode
label_encoder = LabelEncoder()

# List of categorical columns to encode
categorical_columns = ['STRT_CODE1', 'StreetName', 'STRT_CODE2']

# Apply label encoding to each column
for col in categorical_columns:
    buildings[col] = label_encoder.fit_transform(buildings[col])
    
for col in buildings.columns:
    if pd.api.types.is_numeric_dtype(buildings[col]):
        buildings[col].fillna(buildings[col].mean())
buildings = buildings.dropna()

In [5]:
# load municipal buildings
gdb_file = gdb_files[2]  # Modify if you want to choose a different .gdb
print(gdb_file)

# List the layers in the selected .gdb
layers = fiona.listlayers(gdb_file)
print("Layers in the selected GDB:", layers)

# Choose a specific layer within the .gdb
textures_layer = layers[0]  # Modify if you want to choose a different layer

# Load the specific layer
gdf = gpd.read_file(gdb_file, layer=textures_layer)

Michaels_Data\מרקמים\commondata\myproject16.gdb
Layers in the selected GDB: ['סיווג_מרקמים']


In [6]:
textures = gdf.to_crs(local_crs)

In [14]:
# generates a random height for each building 
import numpy as np

# Generate random positive numbers
random_numbers = np.random.rand(len(buildings))

# Create a new column in the buildings DataFrame
buildings['building_height'] = random_numbers

In [15]:
# Calculate Building Height
# We need to have a 'height_col' or a similar column to calculate this

# Calculate Volume, requires floor_area and height
# Assuming height and area have been calculated and are stored in 'height' and 'floor_area' columns
buildings['floor_area'] = momepy.floor_area(buildings['Shape_Area'],buildings['building_height'])
buildings['volume'] = momepy.volume(buildings['Shape_Area'], buildings['building_height'])
buildings['form_factor'] = momepy.form_factor(buildings, buildings['building_height'])

In [7]:
# Basic geometric properties
buildings['perimeter'] = buildings.geometry.length
buildings['shape_index'] = momepy.shape_index(buildings,momepy.longest_axis_length(buildings))
buildings['circular_compactness'] = momepy.circular_compactness(buildings)
buildings['square_compactness'] = momepy.square_compactness(buildings)
buildings['weighted_axis_compactness'] = momepy.compactness_weighted_axis(buildings)
buildings['convexity'] = momepy.convexity(buildings)
buildings['courtyard_area'] = momepy.courtyard_area(buildings)
buildings['courtyard_index'] = momepy.courtyard_index(buildings)
# buildings['corners'] = momepy.corners(buildings, include_interiors=True)   # include_interiors=False works only for polygons not multipolygons
buildings['fractal_dimension'] = momepy.fractal_dimension(buildings)
buildings['facade_ratio'] = momepy.facade_ratio(buildings)
# buildings_centroid_corner_distance = momepy.centroid_corner_distance(buildings)
# buildings['centroid_corner_distance_mean'] = buildings_centroid_corner_distance['mean']
# buildings['centroid_corner_distance_std'] = buildings_centroid_corner_distance['std']

In [9]:
# More complex morphological metrics
buildings['orientation'] = momepy.orientation(buildings)
buildings['longest_axis_length'] = momepy.longest_axis_length(buildings)
buildings['equivalent_rectangular_index'] = momepy.equivalent_rectangular_index(buildings)
buildings['elongation'] = momepy.elongation(buildings)
# buildings['linearity'] = momepy.linearity(buildings) # does not work on buildings just on streets
buildings['rectangularity'] = momepy.rectangularity(buildings)
# buildings['squareness'] = momepy.squareness(buildings, include_interiors=True)   # without interiors works only for polygons not multipolygons
buildings['shared_walls_length'] = momepy.shared_walls(buildings) / buildings['perimeter'] 
buildings['perimeter_wall'] = momepy.perimeter_wall(buildings) # TODO: check for possible parameters 
buildings['perimeter_wall'] = buildings['perimeter_wall'].fillna(0)

In [10]:
# Metrics related to building adjacency and Graph
from libpysal import graph
delaunay = graph.Graph.build_triangulation(buildings.centroid,coplanar='clique').assign_self_weight()
orientation = momepy.orientation(buildings)
buildings['alignment'] = momepy.alignment(orientation, delaunay)

knn15 = graph.Graph.build_knn(buildings.centroid, k=15, coplanar='clique')  # adjust k if needed   
contiguity = graph.Graph.build_contiguity(buildings)
buildings['adjacency'] = momepy.building_adjacency(contiguity,knn15) #TODO: check for queen1 and queen3
buildings['mean_interbuilding_distance'] = momepy.mean_interbuilding_distance(buildings, delaunay, knn15) # TODO: check for queen1 and queen3
buildings['neighbour_distance'] = momepy.neighbor_distance(buildings, delaunay)
buildings['courtyards_num'] = momepy.courtyards(buildings, contiguity) # Calculate the number of courtyards within the joined structure

C:\Users\ameer\PycharmProjects\DSSG\.venv\Lib\site-packages\momepy\functional\_distribution.py:285: UserWarning: The indices of the two GeoSeries are different.
  (geoms.distance(geometry.geometry, align=True)).groupby(level=0).mean()


In [11]:
# This is a modified version of the momepy.buffered_limit function that allows for adaptive buffer calculation on our data
from libpysal import graph
from packaging.version import Version
import numpy as np

def buffered_limit(gdf,buffer: float | str = 100,min_buffer: float = 0,max_buffer: float = 100,**kwargs,):
   
    if buffer == "adaptive":
        
        gabriel = graph.Graph.build_triangulation(gdf.centroid, "gabriel", kernel="identity", coplanar='clique')
        max_dist = gabriel.aggregate("max")
        buffer = np.clip(max_dist / 2 + max_dist * 0.1, min_buffer, max_buffer).values

    elif not isinstance(buffer, int | float):
        raise ValueError("`buffer` must be either 'adaptive' or a number.")

    GPD_GE_10 = Version(gpd.__version__) >= Version("1.0dev")
    return (
        gdf.buffer(buffer, **kwargs).union_all()
        if GPD_GE_10
        else gdf.buffer(buffer, **kwargs).unary_union
    )

In [12]:
# metrics related to tessellation
limit = buffered_limit(buildings, buffer = 'adaptive')
tessellation = momepy.morphological_tessellation(buildings, clip=limit) # need adjustment
blg_orient = momepy.orientation(buildings)
tess_orient = momepy.orientation(tessellation)
buildings['cell_orientation'] = momepy.cell_alignment(blg_orient, tess_orient)
buildings['num_of_neighbours'] = momepy.neighbors(tessellation, contiguity, weighted=True)

In [ ]:
# the following metrics caculate the diversity index of building types, it's possible to override the binning method (refer to the full documentation)
import inequality  # required for Theil index
knn5 = graph.Graph.build_knn(buildings.centroid, k=5)
for col in buildings.columns:
    if pd.api.types.is_numeric_dtype(buildings[col]):
        new_columns = {}
        print(f'Calculating metrics for {col}')
        buildings[f'{col}_shannon'] = momepy.shannon(buildings[col], knn5)
        buildings[f'{col}_simpson'] = momepy.simpson(buildings[col], knn5)
        buildings[f'{col}_theil'] = momepy.theil(buildings[col], knn5)
        buildings[f'{col}_values_range'] = momepy.values_range(buildings[col], knn5)
        buildings[f'{col}_mean_deviation'] = momepy.mean_deviation(buildings[col], knn5)
        
        # Concatenate the new columns to the buildings DataFrame all at once
        buildings = pd.concat([buildings, pd.DataFrame(new_columns)], axis=1)
        buildings = buildings.copy()


In [1]:
buildings.to_file('buildings.gdb', driver='FileGDB')
tessellation.to_file('buildings.gdb', driver='FileGDB')

NameError: name 'buildings' is not defined

# FOR TESTING (irrelevant)

Check leading metrics for buildings

In [ ]:
# Identify columns that contain NaN values
columns_with_nan = buildings.columns[buildings.isna().any()].tolist()
if len(columns_with_nan) > 0:
    raise ValueError(f"Columns with NaN values: {columns_with_nan}")

In [ ]:
from helper_functions import find_overlapping

buildings = find_overlapping(buildings, textures, column_to_add="סיווג_מרקמים_12042024")

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Assuming `df` is your DataFrame and `city` is the column you want to encode
label_encoder = LabelEncoder()

# Fit and transform the 'city' column
buildings['texture'] = label_encoder.fit_transform(buildings['containing_polygon_סיווג_מרקמים_12042024'])
buildings.drop(columns=['containing_polygon_סיווג_מרקמים_12042024'], inplace=True)

In [ ]:
from helper_functions import find_group_leading_metrics, find_overall_leading_metrics

gdf = gpd.GeoDataFrame(buildings, geometry='geometry')
classification_column = 'texture'
leading_metrics_per_group = find_group_leading_metrics(buildings, classification_column, n_components=1)

classification_column = 'texture'
top_metrics_per_group = find_overall_leading_metrics(gdf, classification_column)

# Display the results

In [23]:
graph = momepy.gdf_to_nx(streets)
graph = momepy.node_degree(graph)
graph = momepy.closeness_centrality(graph, radius=400, distance="mm_len")
graph = momepy.meshedness(graph, radius=400, distance="mm_len")
nodes, edges = momepy.nx_to_gdf(graph)

C:\Users\ameer\PycharmProjects\DSSG\.venv\Lib\site-packages\momepy\utils.py:352: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


  0%|          | 0/8707 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:

queen_1 = libpysal.graph.Graph.build_contiguity(tessellation, rook=False)
queen_3 = queen_1.higher_order(3)
buildings_q1 = libpysal.graph.Graph.build_contiguity(buildings, rook=False)

In [ ]:

buildings["interbuilding_distance"] = momepy.mean_interbuilding_distance(
    buildings, queen_1, queen_3
)
buildings["adjacency"] = momepy.building_adjacency(buildings_q1, queen_3)

In [ ]:

merged = tessellation.merge(
    edges.drop(columns="geometry"),
    left_on="edge_index",
    right_index=True,
    how="left",
)
merged = merged.merge(
    nodes.drop(columns="geometry"),
    left_on="node_index",
    right_index=True,
    how="left",
)

In [ ]:

percentiles = []
for column in merged.columns.drop(
    [
        "street_index",
        "node_index",
        "edge_index",
        "nodeID",
        "mm_len",
        "node_start",
        "node_end",
        "geometry",
    ]
):
    perc = momepy.percentile(merged[column], queen_3)
    perc.columns = [f"{column}_" + str(x) for x in perc.columns]
    percentiles.append(perc)

In [21]:

percentiles_joined = pandas.concat(percentiles, axis=1)
percentiles_joined.head()

fig, ax = plt.subplots(1, 2, figsize=(24, 12))

tessellation.plot("convexity", ax=ax[0], scheme="natural_breaks", legend=True)
merged.plot(
    percentiles_joined["convexity_50"].values,
    ax=ax[1],
    scheme="natural_breaks",
    legend=True,
)

ax[0].set_axis_off()
ax[1].set_axis_off()

standardized = (
    percentiles_joined - percentiles_joined.mean()
) / percentiles_joined.std()
standardized.head()

cgram = Clustergram(range(1, 12), n_init=10, random_state=42)
cgram.fit(standardized.fillna(0))

show(cgram.bokeh())

C:\Users\ameer\PycharmProjects\DSSG\.venv\Lib\site-packages\momepy\utils.py:352: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


  0%|          | 0/8707 [00:00<?, ?it/s]

KeyboardInterrupt: 